In [31]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
import string
import itertools

In [32]:
alphabet1, alphabet2 = list(string.ascii_uppercase), list(string.ascii_uppercase)
combination = ['WT']
for a1 in alphabet1:
    for a2 in alphabet2:
        if a1+a2 != 'WT':
            combination.append(a1+a2)

In [33]:
# 데이터 로드
train = pd.read_csv("/content/train.csv", header=0)
test = pd.read_csv("/content/test.csv", header=0)

# SUBCLASS 라벨 인코딩
le_subclass = LabelEncoder()
train['SUBCLASS'] = le_subclass.fit_transform(train['SUBCLASS'])
for i, label in enumerate(le_subclass.classes_):
    print(f"원래 레이블: {label}, 변환된 숫자: {i}")

X = train.drop(columns=['SUBCLASS', 'ID']).copy()
y_subclass = train['SUBCLASS']


# 훈련 데이터 전처리
def preprocessing_train(element):
    # 띄어쓰기로 분리되어있는 element는 리스트로 쪼개기
    if isinstance(element, str) and ' ' in element:
        element = element.split()

    # 값이 한 개(문자열)인 경우
    if isinstance(element, str):
        if element[0] + element[-1] not in combination:
            combination.append(element[0]+element[-1])  ## 알파벳이 아닌 * 와 같은 문자도 있었음
        return np.where(element[0] == element[-1], 0, combination.index(element[0] + element[-1]))

    # 값이 여러개(리스트)인 경우
    # elif isinstance(element, list):
    #     return [preprocessing_train(el).item() for el in element]

    elif isinstance(element, list):
         return preprocessing_train(element[1])


# applymap으로 각 요소에 대해 함수 적용
X_prep = X.applymap(preprocessing_train)

# 변환된 데이터 확인
print(X_prep)
print('업데이트된 조합표')
print(combination)
X_prep.to_csv('preprocessing_data.csv')

원래 레이블: ACC, 변환된 숫자: 0
원래 레이블: BLCA, 변환된 숫자: 1
원래 레이블: BRCA, 변환된 숫자: 2
원래 레이블: CESC, 변환된 숫자: 3
원래 레이블: COAD, 변환된 숫자: 4
원래 레이블: DLBC, 변환된 숫자: 5
원래 레이블: GBMLGG, 변환된 숫자: 6
원래 레이블: HNSC, 변환된 숫자: 7
원래 레이블: KIPAN, 변환된 숫자: 8
원래 레이블: KIRC, 변환된 숫자: 9
원래 레이블: LAML, 변환된 숫자: 10
원래 레이블: LGG, 변환된 숫자: 11
원래 레이블: LIHC, 변환된 숫자: 12
원래 레이블: LUAD, 변환된 숫자: 13
원래 레이블: LUSC, 변환된 숫자: 14
원래 레이블: OV, 변환된 숫자: 15
원래 레이블: PAAD, 변환된 숫자: 16
원래 레이블: PCPG, 변환된 숫자: 17
원래 레이블: PRAD, 변환된 숫자: 18
원래 레이블: SARC, 변환된 숫자: 19
원래 레이블: SKCM, 변환된 숫자: 20
원래 레이블: STES, 변환된 숫자: 21
원래 레이블: TGCT, 변환된 숫자: 22
원래 레이블: THCA, 변환된 숫자: 23
원래 레이블: THYM, 변환된 숫자: 24
원래 레이블: UCEC, 변환된 숫자: 25
      A2M  AAAS  AADAT  AARS1  ABAT  ABCA1  ABCA2  ABCA3  ABCA4  ABCA5  ...  \
0       0     0      0      0     0      0      0      0      0      0  ...   
1       0     0      0      0     0      0      0      0      0      0  ...   
2       0     0      0      0     0      0      0      0      0      0  ...   
3       0     0      0      0     0      0   

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_prep, y_subclass, test_size=0.2, random_state=42)

# XGBoost 모델 설정
model = xgb.XGBClassifier(
    n_estimators=1000,
    learning_rate=0.1,
    max_depth=7,
    random_state=42,
    eval_metric='mlogloss',
    reg_lambda=15,
    subsample=0.5,
    colsample_bytree=0.5
)

# 검증 세트를 훈련 세트로 설정
eval_set = [(X_val, y_val)]

# 모델 학습
model.fit(X_train, y_train, eval_set=eval_set, verbose=100)

# 학습된 모델을 저장
model_filename = './model_saved/xgboost_model.bin'
model.save_model(model_filename)
print(f"모델이 {model_filename}에 저장되었습니다.")

[0]	validation_0-mlogloss:3.19005
[100]	validation_0-mlogloss:2.15833
[200]	validation_0-mlogloss:2.06650
[300]	validation_0-mlogloss:2.04025
[400]	validation_0-mlogloss:2.03263
[500]	validation_0-mlogloss:2.03787
[600]	validation_0-mlogloss:2.04665
[700]	validation_0-mlogloss:2.05637
[800]	validation_0-mlogloss:2.07306


In [ ]:
# 테스트 데이터 전처리
def preprocessing_test(element):
    # 띄어쓰기로 분리되어있는 element는 리스트로 쪼개기
    if isinstance(element, str) and ' ' in element:
        element = element.split()

    # 값이 한 개(문자열)인 경우
    if isinstance(element, str):
        if element[0] + element[-1] not in combination:
            combination.append(element[0] + element[-1])  ## 알파벳이 아닌 *와 같은 문자도 있었음
        return np.where(element[0] == element[-1], 0, combination.index(element[0] + element[-1]))

    # 값이 여러개(리스트)인 경우
    # elif isinstance(element, list):
    #     return [preprocessing_test(el).item() for el in element]

    elif isinstance(element, list):
         return preprocessing_test(element[0])

# 테스트 데이터 전처리 적용
test_X = test.drop(columns=['ID']).copy()
X_encoded_test = test_X.applymap(preprocessing_test)

# 예측
predictions = model.predict(X_encoded_test)

# 원래 레이블로 변환
original_labels = le_subclass.inverse_transform(predictions)

# 제출 파일
submission = pd.read_csv("sample_submission.csv")
submission["SUBCLASS"] = original_labels
submission.to_csv('./submission_dump/XGBoost_submission.csv', encoding='UTF-8-sig', index=False)